_Camille Avestruz (UChicago)_, _Alex Malz (NYU)_, & Others

Our understanding of how folks currently use this code is based on the [Analytic Bias Estimate (ABE)](https://github.com/LSSTDESC/clmassmod/blob/master/notebooks/Analytic_bias_estimate.ipynb) and [Statistical Power Check (SPC)](https://github.com/LSSTDESC/clmassmod/blob/master/notebooks/Statistical%20Power%20Check.ipynb) demos.

This is a wishlist for how we'd like to use `clmassmod`.  We will refactor the code to make this notebook run!

There was a lot of other stuff here but we think it was more about simulating mock data, not the analysis pipeline we want to work on.  It can be found in commit `a103463`.

In [ ]:
import clmassmod as clmm

if you want to do this in bins, define them out here however you want and then run the below steps per bin

`model_info` would be a dict with the name of the model and its parameters to support comparison of models, read in from a config file by the user/script outside the CLMassMod object.

In [ ]:
massmod = clm.CLMassMod(model_info)

`in_data` includes mass map and more

`calc_params` is like curvefit vs pymc vs whatever

`the_fit` is the object produced by the fitter, like the pymc MC object, with properties like diagnostics already in it

`out_data` is pdfs or mles

In [ ]:
out_data, the_fit = massmod.individual_profile_fits(in_data, calc_params)
    # parallelization option in calc_params
    # writes one file per halo

Write out the individual profile fits as insurance against computational instability

In [ ]:
massmod.write(directory_name)

bin your clusters as you see fit and read in for one set of them, so imagine everything below here is in a loop or in parallel for many bins

`bin_directory_name` would be bin-specific

In [ ]:
massmod = clm.CLMassMod(model_info)

Read in the individual profile fits, so this can be done totally independently

In [ ]:
out_data = massmod.read(bin_directory_name)

again, `calc_params` is a dict including the name of the fitting method and any tunable kw parameters it needs

In [ ]:
clust_mass_dist = massmod.ensemble_fit(calc_params)

`truth` is the masses from simulations

In [ ]:
likelihoods, posteriors, mass_dist_bias_samps, mass_dist_scatter_samps = massmod.compare(truth)